In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = True if value == 'True' else False

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())


        big_data = big_data.append(d, ignore_index=True)

# files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
# i = 0
# for filename in files:
#     #print(f'parsing : {filename}')
#     json_file = filename.replace('backtest_result.txt','live_config.json')
#     with open(json_file) as f:
#         live_config = json.load(f)
#         config_type = live_config['config_name']
#
#     with open(filename, 'r') as file:
#         data = file.read().split('\n')
#         d = {'file': filename.replace('backtests/binance_production','')}
#         current_state = 'long'
#         for l in data:
#             l = l.split('|')
#             if len(l) == 4:
#                 key = l[1].strip()
#                 value = l[2].strip()
#
#                 if key == 'Passivbot Version':
#                     d['v'] = value
#                 if key == 'Symbol':
#                     d['s'] = value
#
#                 if key == 'Long':
#                     current_state = 'long'
#                     d['long'] = bool(value) or False
#                 if key == 'Short':
#                     current_state = 'short'
#                     d['short'] = bool(value) or False
#
#                 if key == 'Total gain' and current_state == 'long':
#                     d['tgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'long':
#                     d['adgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'long':
#                     d['bnkrl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'long':
#                     d['ebrl'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'long':
#                     d['maxsl'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'long':
#                     d['pamaxl'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'long':
#                     d['pastdl'] = float(value.strip())
#
#
#                 if key == 'Total gain' and current_state == 'short':
#                     d['tgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'short':
#                     d['adgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'short':
#                     d['bnkrs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'short':
#                     d['ebrs'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'short':
#                     d['maxss'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'short':
#                     d['pamaxs'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'short':
#                     d['pastds'] = float(value.strip())
#
#         d['prod'] = True
#         big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_608932/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_608932/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_608932/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_608932/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod
0,\1000SHIBUSDT\plots\2022-04-03T194816\backtest...,5.5,1000SHIBUSDT,True,52.53,0.1780,0.066776,0.644511,100.00,0.8613,12.442375,True,-96.020,-1.88000,0.146280,0.802360,5.447,0.10870,155.190833,NaN
1,\1000SHIBUSDT\plots\2022-04-03T204145\backtest...,5.5,1000SHIBUSDT,True,26.73,0.0850,0.070106,0.830417,100.00,0.9153,6.572917,True,-95.820,-1.88000,0.148225,0.803949,5.404,0.09669,155.191667,NaN
2,\1000SHIBUSDT\plots\2022-04-06T180851\backtest...,5.5,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.00,0.7673,12.442375,True,-91.820,-1.64000,0.124514,0.747041,5.959,0.08869,180.079167,True
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.00,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
4,\1000XECUSDT\plots\2022-04-06T181135\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.00,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,\ZENUSDT\plots\2022-04-07T123822\backtest_resu...,5.5,ZENUSDT,True,71.30,0.2010,0.128784,0.968012,100.00,0.8159,14.339583,True,20.720,0.10100,0.109054,0.420619,49.720,0.56430,39.212500,True
205,\ZENUSDT\plots\2022-04-07T234033\backtest_resu...,5.5,ZENUSDT,True,61.20,0.1830,0.107443,0.877769,95.61,0.7091,11.806250,True,28.960,0.33000,0.140844,0.613587,18.190,0.22950,49.320833,False
206,\ZENUSDT\plots\2022-04-08T000448\backtest_resu...,5.5,ZENUSDT,True,78.47,0.2110,0.129356,0.981507,100.00,0.8117,14.342375,True,8.361,0.19300,0.132544,0.571631,19.640,0.25670,47.980000,False
207,\ZILUSDT\plots\2022-04-07T124653\backtest_resu...,5.5,ZILUSDT,True,11.86,0.0150,0.213304,1.171540,100.00,0.8399,35.809042,True,-90.810,-1.64000,0.128495,0.776683,5.828,0.07862,33.732625,True


Now filter the dataframe

In [4]:
# allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
#                    "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
#                    "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
#                    "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
#                    "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
#                    "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
#                    "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
#                    "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
#                    "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
#                    "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
#                    "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
#                    "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
#                    "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
#                    "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
#                    "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
#                    "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
#                    "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
#                    "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
#                    "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
#                    "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = [
    "OMG","DUSK","GTC"
]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .85)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,...,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod,adgt_%
69,\DUSKUSDT\plots\2022-04-07T001453\backtest_res...,5.5,DUSKUSDT,True,21.41,0.0328,0.690817,2.691040,100.0,0.7918,...,True,37.37,0.2220,0.110620,0.458401,100.000,0.79310,21.731250,True,0.2548
70,\DUSKUSDT\plots\2022-04-09T140544\backtest_res...,5.5,DUSKUSDT,True,25.21,0.2920,0.166748,0.805989,100.0,0.7017,...,True,36.33,0.2260,0.102230,0.440883,100.000,0.79950,22.862500,False,0.5180
71,\DUSKUSDT\plots\2022-04-09T140914\backtest_res...,5.5,DUSKUSDT,True,18.54,0.1720,0.231191,1.222280,100.0,0.8377,...,True,18.05,0.2030,0.112300,0.437698,100.000,0.79690,19.933333,False,0.3750
97,\GTCUSDT\plots\2022-04-07T032352\backtest_resu...,5.5,GTCUSDT,True,34.16,0.1130,0.228586,1.021810,100.0,0.8536,...,True,-91.37,-1.6400,0.142883,0.760463,5.717,0.10870,129.768750,True,-1.5270
98,\GTCUSDT\plots\2022-04-09T153137\backtest_resu...,5.5,GTCUSDT,True,86.88,0.2430,0.116216,0.659542,100.0,0.8256,...,True,-91.21,-1.6400,0.134724,0.752127,5.835,0.08835,129.366667,False,-1.3970
99,\GTCUSDT\plots\2022-04-09T153349\backtest_resu...,5.5,GTCUSDT,True,45.17,0.1400,0.226968,1.027650,100.0,0.8609,...,True,-91.43,-1.6400,0.127501,0.743179,5.944,0.09973,129.236250,False,-1.5000
136,\OMGUSDT\plots\2022-04-07T071804\backtest_resu...,5.5,OMGUSDT,True,37.06,0.1150,0.188849,0.984657,100.0,0.8703,...,True,22.56,0.0486,0.119473,0.526916,80.570,0.60590,27.884042,True,0.1636
137,\OMGUSDT\plots\2022-04-09T140231\backtest_resu...,5.5,OMGUSDT,True,67.29,0.1930,0.080248,0.654878,100.0,0.8145,...,True,19.53,0.0455,0.132583,0.520145,80.400,0.60850,30.813208,False,0.2385
138,\OMGUSDT\plots\2022-04-09T140744\backtest_resu...,5.5,OMGUSDT,True,50.36,0.1450,0.084642,0.857505,100.0,0.8802,...,True,-91.52,-1.6400,0.231068,0.753377,5.961,0.07838,185.663333,False,-1.4950


In [5]:
df.drop_duplicates(
    subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file', 'prod', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_608932/2052413415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
69,\DUSKUSDT\plots\2022-04-07T001453\backtest_res...,True,DUSKUSDT,0.2548,0.0328,0.2220,100.0,100.000,0.7918,0.79310,77.917917,2.691040,0.690817,21.731250,0.458401,0.110620
70,\DUSKUSDT\plots\2022-04-09T140544\backtest_res...,False,DUSKUSDT,0.5180,0.2920,0.2260,100.0,100.000,0.7017,0.79950,17.735417,0.805989,0.166748,22.862500,0.440883,0.102230
71,\DUSKUSDT\plots\2022-04-09T140914\backtest_res...,False,DUSKUSDT,0.3750,0.1720,0.2030,100.0,100.000,0.8377,0.79690,21.250000,1.222280,0.231191,19.933333,0.437698,0.112300
97,\GTCUSDT\plots\2022-04-07T032352\backtest_resu...,True,GTCUSDT,-1.5270,0.1130,-1.6400,100.0,5.717,0.8536,0.10870,26.844458,1.021810,0.228586,129.768750,0.760463,0.142883
98,\GTCUSDT\plots\2022-04-09T153137\backtest_resu...,False,GTCUSDT,-1.3970,0.2430,-1.6400,100.0,5.835,0.8256,0.08835,12.261792,0.659542,0.116216,129.366667,0.752127,0.134724
99,\GTCUSDT\plots\2022-04-09T153349\backtest_resu...,False,GTCUSDT,-1.5000,0.1400,-1.6400,100.0,5.944,0.8609,0.09973,26.819458,1.027650,0.226968,129.236250,0.743179,0.127501
136,\OMGUSDT\plots\2022-04-07T071804\backtest_resu...,True,OMGUSDT,0.1636,0.1150,0.0486,100.0,80.570,0.8703,0.60590,53.311667,0.984657,0.188849,27.884042,0.526916,0.119473
137,\OMGUSDT\plots\2022-04-09T140231\backtest_resu...,False,OMGUSDT,0.2385,0.1930,0.0455,100.0,80.400,0.8145,0.60850,14.779875,0.654878,0.080248,30.813208,0.520145,0.132583
138,\OMGUSDT\plots\2022-04-09T140744\backtest_resu...,False,OMGUSDT,-1.4950,0.1450,-1.6400,100.0,5.961,0.8802,0.07838,7.366667,0.857505,0.084642,185.663333,0.753377,0.231068
